In [11]:
# Some imports, we are not gong to use all the imports in this workbook but in subsequent workbooks we surely will.
import os
import time

import numpy as np
import pandas as pd

from toolz import pipe as p
import tensorflow as tf


In [12]:
import model_runs
import models as models
import utils as u

from importlib import reload
reload(u)
reload(model_runs)
reload(models)

<module 'models' from 'C:\\Users\\jod204\\projects\\jo\\samplesize_test\\models.py'>

# Settings and Helpers

In [26]:
# Define some Global Variables
max_features = 20000 # Maximum Number of words we want to include in our dictionary
maxlen = 400 # No of words in question we want to create a sequence with
embed_size = 50# Size of word to vec embedding we are using


def get_log_dir(iteration, samples_per_class):
    return f"output/{samples_per_class}/{iteration}/"


def save_series_gen(log_dir, prefix):
    def save_series(series, name):
        series.to_csv(f'{log_dir}/{prefix}_{name}.csv', index = False, header = False)
    
    return save_series


def log_truths(log_dir, train_y, val_y, test_y):
    save_series = save_series_gen(log_dir, 'true')
        
    save_series(train_y, 'train')
    save_series(val_y, 'val')
    save_series(test_y, 'test')


def log_iteration_preds(log_dir, model_name,
                  pred_train_y, pred_val_y, pred_test_y,
                  one_train_time):
    def save_arr(arr, name):
        arr.tofile(f'{log_dir}/{model_name}_{name}.csv', sep = ',')
    
    save_arr(pred_train_y, 'pred_train')
    save_arr(pred_val_y, 'pred_val')
    save_arr(pred_test_y, 'pred_test')
    p(one_train_time,
      np.array,
      lambda _: save_arr(_, 'time'))


def run_and_log_model(model, model_name, train_X, train_y, val_X, val_y, test_X, iteration, samples_per_class):
    start_time = time.perf_counter()
    pred_train_y, pred_val_y, pred_test_y = model_runs.train_pred(model, train_X, train_y,
                                          val_X, val_y, test_X, epochs=50)
    end_time = time.perf_counter()
    one_train_time = end_time - start_time
    
    dir_name = get_log_dir(iteration, samples_per_class)
    log_iteration_preds(dir_name, model_name, 
                  pred_train_y, pred_val_y, pred_test_y,
                  one_train_time) 


# Model Execution

In [ ]:
if __name__ == '__main__':
  with tf.device('/device:GPU:0'):
      data = pd.read_csv('./content/pubmed_cr_hep_ctl_abstracts_clean.csv')

      for samples_per_class in range(500, 2500, 500):
          for i in range(0, 1):
              train_X, val_X, test_X, train_y, val_y, test_y, word_index = u.load_and_prec(
                  samples_per_class, max_features, maxlen = maxlen, data = data)
            
              log_dir = get_log_dir(i, samples_per_class)
              os.makedirs(log_dir, exist_ok = True)
            
              log_truths(log_dir, train_y, val_y, test_y)
              
              print(test_X.shape[1])
              sequence_len = test_X.shape[1]
                
              def run_and_log_model_curry(model, model_name):
                run_and_log_model(model, model_name, 
                                  train_X, train_y, val_X, val_y, test_X, 
                                  i, samples_per_class)

              # CNN
              cnn_model_built = models.cnn_model(sequence_len, [3, 4, 5], maxlen, max_features, embed_size, 
                                                 num_filters=200, drop_rate=0.2)
              cnn_model_built.summary()
              run_and_log_model_curry(cnn_model_built, 'cnn')

              # LSTM
              lstm_model = models.model_lstm_du(maxlen, max_features, embed_size)
              lstm_model.summary()
              run_and_log_model_curry(lstm_model, 'lstm')

              # HAN
              han_model = models.model_lstm_atten(maxlen, max_features, embed_size)
              han_model.summary()
              run_and_log_model_curry(han_model, 'han')


1000
500
500
X shape :  (1000,)
y shape :  (1000,)
400
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 400, 50)      1000000     input_2[0][0]                    
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 400, 50, 1)   0           embedding_2[0][0]                
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 398, 1, 200)  30200       reshape_3[0][0]                  
______________________________________________________

603/603 [==============================] - 0s 166us/step - loss: 0.2160 - acc: 0.9818 - val_loss: 0.3218 - val_acc: 0.9254
Epoch 36/50
603/603 [==============================] - 0s 192us/step - loss: 0.2084 - acc: 0.9834 - val_loss: 0.3153 - val_acc: 0.9254
Epoch 37/50
603/603 [==============================] - 0s 192us/step - loss: 0.2033 - acc: 0.9834 - val_loss: 0.3118 - val_acc: 0.9254
Epoch 38/50
603/603 [==============================] - 0s 171us/step - loss: 0.1956 - acc: 0.9834 - val_loss: 0.3102 - val_acc: 0.9254
Epoch 39/50
603/603 [==============================] - 0s 188us/step - loss: 0.1898 - acc: 0.9834 - val_loss: 0.3065 - val_acc: 0.9254
Epoch 40/50
603/603 [==============================] - 0s 192us/step - loss: 0.1844 - acc: 0.9818 - val_loss: 0.3005 - val_acc: 0.9254
Epoch 41/50
603/603 [==============================] - 0s 192us/step - loss: 0.1799 - acc: 0.9834 - val_loss: 0.2959 - val_acc: 0.9254
Epoch 42/50
603/603 [==============================] - 0s 177us/ste

Train on 603 samples, validate on 67 samples
Epoch 1/50
603/603 [==============================] - 7s 12ms/step - loss: 0.6939 - acc: 0.4693 - val_loss: 0.6928 - val_acc: 0.5522
Epoch 2/50
603/603 [==============================] - 4s 7ms/step - loss: 0.6930 - acc: 0.5158 - val_loss: 0.6942 - val_acc: 0.4478

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 3/50
603/603 [==============================] - 4s 7ms/step - loss: 0.6931 - acc: 0.4959 - val_loss: 0.6943 - val_acc: 0.4478

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 4/50
603/603 [==============================] - 4s 7ms/step - loss: 0.6930 - acc: 0.4959 - val_loss: 0.6940 - val_acc: 0.4478

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 5/50
603/603 [==============================] - 4s 7ms/step - loss: 0.6929 - acc: 0.4959 - val_loss: 0.6938 - val_acc: 0.4478

Epoch 00005: ReduceLROnPlateau reducing learning r

1206/1206 [==============================] - 0s 179us/step - loss: 0.1919 - acc: 0.9784 - val_loss: 0.3342 - val_acc: 0.9179
Epoch 27/50
1206/1206 [==============================] - 0s 179us/step - loss: 0.1843 - acc: 0.9793 - val_loss: 0.3305 - val_acc: 0.9179
Epoch 28/50
1206/1206 [==============================] - 0s 179us/step - loss: 0.1773 - acc: 0.9818 - val_loss: 0.3273 - val_acc: 0.9179
Epoch 29/50
1206/1206 [==============================] - 0s 179us/step - loss: 0.1714 - acc: 0.9842 - val_loss: 0.3256 - val_acc: 0.9104
Epoch 30/50
1206/1206 [==============================] - 0s 184us/step - loss: 0.1639 - acc: 0.9859 - val_loss: 0.3238 - val_acc: 0.9179
Epoch 31/50
1206/1206 [==============================] - 0s 179us/step - loss: 0.1590 - acc: 0.9842 - val_loss: 0.3213 - val_acc: 0.9179
Epoch 32/50
1206/1206 [==============================] - 0s 179us/step - loss: 0.1546 - acc: 0.9859 - val_loss: 0.3181 - val_acc: 0.9179
Epoch 33/50
1206/1206 [==============================

1206/1206 [==============================] - 3s 2ms/step - loss: 0.2086 - acc: 0.9030 - val_loss: 0.6548 - val_acc: 0.7687

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.0001.
Epoch 15/50
1206/1206 [==============================] - 3s 2ms/step - loss: 0.1833 - acc: 0.9129 - val_loss: 0.6694 - val_acc: 0.7537
Epoch 00015: early stopping
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 400)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 400, 256)          183296    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 400, 128)          164352    
______________________________________________________

1809/1809 [==============================] - 0s 184us/step - loss: 0.6390 - acc: 0.8612 - val_loss: 0.6268 - val_acc: 0.9055
Epoch 6/50
1809/1809 [==============================] - 0s 178us/step - loss: 0.6009 - acc: 0.9453 - val_loss: 0.5921 - val_acc: 0.9104
Epoch 7/50
1809/1809 [==============================] - 0s 184us/step - loss: 0.5564 - acc: 0.9547 - val_loss: 0.5576 - val_acc: 0.9055
Epoch 8/50
1809/1809 [==============================] - 0s 184us/step - loss: 0.5105 - acc: 0.9569 - val_loss: 0.5225 - val_acc: 0.9055
Epoch 9/50
1809/1809 [==============================] - 0s 185us/step - loss: 0.4632 - acc: 0.9508 - val_loss: 0.4904 - val_acc: 0.9104
Epoch 10/50
1809/1809 [==============================] - 0s 177us/step - loss: 0.4205 - acc: 0.9469 - val_loss: 0.4616 - val_acc: 0.9104
Epoch 11/50
1809/1809 [==============================] - 0s 184us/step - loss: 0.3827 - acc: 0.9464 - val_loss: 0.4364 - val_acc: 0.9104
Epoch 12/50
1809/1809 [==============================] - 

Train on 1809 samples, validate on 201 samples
Epoch 1/50
1809/1809 [==============================] - 7s 4ms/step - loss: 0.6929 - acc: 0.5119 - val_loss: 0.6925 - val_acc: 0.4826
Epoch 2/50
1809/1809 [==============================] - 4s 2ms/step - loss: 0.6910 - acc: 0.5097 - val_loss: 0.6917 - val_acc: 0.4826
Epoch 3/50
1809/1809 [==============================] - 4s 2ms/step - loss: 0.6882 - acc: 0.5064 - val_loss: 0.6884 - val_acc: 0.4826
Epoch 4/50
1809/1809 [==============================] - 4s 2ms/step - loss: 0.6827 - acc: 0.6285 - val_loss: 0.6817 - val_acc: 0.8159
Epoch 5/50
1809/1809 [==============================] - 4s 2ms/step - loss: 0.6729 - acc: 0.8911 - val_loss: 0.6690 - val_acc: 0.7861
Epoch 6/50
1809/1809 [==============================] - 4s 2ms/step - loss: 0.6502 - acc: 0.8811 - val_loss: 0.6329 - val_acc: 0.8458
Epoch 7/50
1809/1809 [==============================] - 4s 2ms/step - loss: 0.5835 - acc: 0.8977 - val_loss: 0.4963 - val_acc: 0.8706
Epoch 8/50
1809